## Imports

In [127]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt # general plotting
import matplotlib as mpl # for figure quality (dpi)
import os

## Connect to IMDb database 

In [130]:
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='Jaiswal',
    auth_plugin='mysql_native_password',
    database='IMDb') 

Database cursor 

In [131]:
mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES;")
tables = mycursor.fetchall()

In [132]:
print('IMDb Database contains the following tables:')
print('--------------------------------------------')
for table in tables:
    print(table[0])

IMDb Database contains the following tables:
--------------------------------------------
alias_attributes
alias_types
aliases
directors
episode_belongs_to
had_role
known_for
name_worked_as
names_
principals
q1
q10
q11
q12
q13
q14
q15
q16
q17
q18
q19
q2
q20
q21
q22
q23
q24
q3
q4
q5
q6
q7
q8
q9
title_genres
title_ratings
titles
writers


# Data Dictionary

In [195]:
data_dict = {
    "title_id": "example3_id",
    "title_type": "Movie",
    "primary_title": "Example Movie",
    "original_title": "Original Example Movie",
    "is_adult": False,
    "start_year": 2022,
    "end_year": None,
    "runtime_minutes": 120,
    "average_rating": 9.0,
    "num_votes": 1,
    "ordering": 1,
    "name_id": "actor123",
    "job_category": "Actor",
    "job": "Lead Actor"
}

# Authentication


In [197]:
import mysql.connector
import pandas as pd

class User:
    def __init__(self, username, is_admin=False):
        self.username = username
        self.is_admin = is_admin

class AuthenticationSystem:
    def __init__(self):
        self.users = {}

    def add_user(self, username, is_admin=False):
        if username not in self.users:
            user = User(username, is_admin)
            self.users[username] = user
            return user
        else:
            print(f"User {username} already exists.")
            return self.users[username]

    def execute_query(self, user, query):
        if "DELETE" in query.upper() and not user.is_admin:
            print(f"User {user.username} does not have permission to execute DELETE queries.")
            return None
        else:
            print(f"Executing query: {query}")
            mycursor.execute(query)
            result = mycursor.fetchall()
            mydb.commit()
            column_names = mycursor.column_names
            return result, column_names
        
    def insert_into_titles(self, user, data_dict):
        if not user.is_admin:
            print(f"User {user.username} does not have permission to execute INSERT queries.")
            return None
        else:
            try:
                insert_titles_query = "INSERT INTO Titles (title_id, title_type, primary_title, original_title, is_adult, start_year, end_year, runtime_minutes) VALUES (%(title_id)s, %(title_type)s, %(primary_title)s, %(original_title)s, %(is_adult)s, %(start_year)s, %(end_year)s, %(runtime_minutes)s)"
                mycursor.execute(insert_titles_query, data_dict)
                mydb.commit()
                insert_ratings_query = "INSERT INTO Title_ratings (title_id, average_rating, num_votes) VALUES (%(title_id)s, %(average_rating)s, %(num_votes)s)"
                mycursor.execute(insert_ratings_query, data_dict)
                # insert_principals_query = "INSERT INTO Principals (title_id, ordering, name_id, job_category, job) VALUES (%(title_id)s, %(ordering)s, %(name_id)s, %(job_category)s, %(job)s)"
                # mycursor.execute(insert_principals_query, data_dict)
                mydb.commit()
                print("Insertion successful!")
            except Exception as e:
                print(f"Error: {e}")

    def delete_from_titles(self, user, title_id):
        if not user.is_admin:
            print(f"User {user.username} does not have permission to execute DELETE queries.")
            return None
        else:
            try:
                delete_ratings_query = "DELETE FROM Title_ratings WHERE title_id = %s"
                mycursor.execute(delete_ratings_query, (title_id,))
                mydb.commit()
                delete_titles_query = "DELETE FROM Titles WHERE title_id = %s"
                mycursor.execute(delete_titles_query, (title_id,))
                # delete_query = "DELETE FROM Titles WHERE title_id = %s"
                # mycursor.execute(delete_query, (title_id,))
                mydb.commit()
                print("Deletion successful!")
            except Exception as e:
                print(f"Error: {e}")

auth_system = AuthenticationSystem()
role = input("Enter your role (user or admin): ")
current_user = auth_system.add_user("user", is_admin=(role.lower() == 'admin'))

In [196]:
# data_tuple = ('example_id', 'Movie', 'Example Movie', 'Original Example Movie', False, 2022, None, 120)
auth_system.insert_into_titles(current_user, data_dict)

Insertion successful!


In [198]:
title_id_to_delete = 'example2_id'
auth_system.delete_from_titles(current_user, title_id_to_delete)

Deletion successful!


### Rate a Movie

In [184]:
tup = ('9.0', 'example_id')
query = f"""UPDATE Title_ratings
    SET average_rating = (average_rating * num_votes + {float(tup[0])}) / (num_votes + 1),
        num_votes = num_votes + 1
    WHERE title_id = '{tup[1]}';"""

result, column_names = auth_system.execute_query(current_user, query)

Executing query: UPDATE Title_ratings
    SET average_rating = (average_rating * num_votes + 9.0) / (num_votes + 1),
        num_votes = num_votes + 1
    WHERE title_id = 'example_id';


In [156]:
query_options = {
    1: "SELECT * FROM Q1;",
    2: "SELECT * FROM Q2;",
    3: "SELECT * FROM Q3;",
    4: "SELECT * FROM Q4 LIMIT 10;",
    5: "SELECT * FROM Q5;",
    6: "SELECT * FROM Q6;",
    7: "SELECT * FROM Q7;",
    8: "SELECT * FROM Q8;",
    9: "SELECT * FROM Q9;",
    10: "SELECT * FROM Q10;",
    11: "SELECT * FROM Q11;",
    12: "SELECT * FROM Q12;",
    13: "SELECT * FROM Q13;",
    14: "SELECT * FROM Q14;",
    15: "SELECT * FROM Q15 LIMIT 15;",
    16: "SELECT * FROM Q16;",
    17: "SELECT * FROM Q17;",
    18: "SELECT * FROM Q18;",
    19: "SELECT * FROM Q19;",
    20: "SELECT * FROM Q20;",
    21: "SELECT * FROM Q21;",
    22: "SELECT * FROM Q22;",
    23: "SELECT * FROM Q23;",
    24: "SELECT * FROM Q24;"

}

print("Choose a query to execute:")
query_choice = int(input("Enter the query number: "))
result, column_names = auth_system.execute_query(current_user, query_options.get(query_choice))


Choose a query to execute:
Executing query: SELECT * FROM Q1;


In [157]:
if result is not None:
    df = pd.DataFrame(result, columns=column_names)
    print(df)

  title_type  Count
0      Movie      1
1      movie      9
2      short      1
3  tvEpisode      7
4   tvSeries      4


## Visualising the ratings of the tv show "The X-Files"

What is the average rating of each episode of The X-Files? 

In [136]:
Query1 = """SELECT E.season_number, E.episode_number, T2.primary_title, R.average_rating
FROM Titles AS T1, Titles AS T2, Episode_belongs_to AS E, Title_ratings AS R
WHERE T1.primary_title = 'Breaking Bad'
AND T1.title_type = 'tvSeries'
AND T1.title_id = E.parent_tv_show_title_id
AND T2.title_type = 'tvEpisode'
AND T2.title_id = E.episode_title_id
AND T2.title_id = R.title_id
ORDER BY E.season_number, E.episode_number;"""

In [137]:
result, column_names = auth_system.execute_query(current_user, Query1)

Executing query: SELECT E.season_number, E.episode_number, T2.primary_title, R.average_rating
FROM Titles AS T1, Titles AS T2, Episode_belongs_to AS E, Title_ratings AS R
WHERE T1.primary_title = 'Breaking Bad'
AND T1.title_type = 'tvSeries'
AND T1.title_id = E.parent_tv_show_title_id
AND T2.title_type = 'tvEpisode'
AND T2.title_id = E.episode_title_id
AND T2.title_id = R.title_id
ORDER BY E.season_number, E.episode_number;
   season_number  episode_number primary_title  average_rating
0              1               1           Ep1             9.0
1              2               2           Ep2             9.5


In [138]:
if result is not None:
    df = pd.DataFrame(result, columns=column_names)
    print(df)

   season_number  episode_number primary_title  average_rating
0              1               1           Ep1             9.0
1              2               2           Ep2             9.5


How many episodes were there in Breaking Bad per season? And what was the average of the average episode ratings for each season?

In [139]:
Query2 = """SELECT Q22.season_number, COUNT(*) AS Number_of_episodes, AVG(Q22.average_rating) AS Average_of_ep_average_ratings
FROM Q22
GROUP BY Q22.season_number
ORDER BY Q22.season_number;"""

In [142]:
result, column_names = auth_system.execute_query(current_user, Query2)

Executing query: SELECT Q22.season_number, COUNT(*) AS Number_of_episodes, AVG(Q22.average_rating) AS Average_of_ep_average_ratings
FROM Q22
GROUP BY Q22.season_number
ORDER BY Q22.season_number;


In [143]:
if result is not None:
    df = pd.DataFrame(result, columns=column_names)
    print(df)

   season_number  Number_of_episodes  Average_of_ep_average_ratings
0              1                   1                            9.0
1              2                   1                            9.5


## Genres 

### What genres are there? How many movies are there in each genre?

In [87]:
Query3 ="""SELECT G.genre, COUNT(G.genre) AS Count
FROM Title_genres AS G, Titles AS T
WHERE T.title_id = G.title_id
AND T.title_type = 'movie'
GROUP BY genre
ORDER BY Count DESC;"""

In [144]:
result, column_names = auth_system.execute_query(current_user, Query3)

Executing query: SELECT G.genre, COUNT(G.genre) AS Count
FROM Title_genres AS G, Titles AS T
WHERE T.title_id = G.title_id
AND T.title_type = 'movie'
GROUP BY genre
ORDER BY Count DESC;


In [145]:
if result is not None:
    df = pd.DataFrame(result, columns=column_names)
    print(df)

           genre  Count
0       Action\r      3
1        Short\r      2
2  Documentary\r      1
3    Documentary      1
4       Comedy\r      1
5      Romance\r      1
6        Drama\r      1


### How many movies are made in each genre each year?

In [146]:
Query4 ="""SELECT T.start_year, G.genre, COUNT(DISTINCT T.title_id) AS Number_of_movies
FROM Titles AS T, Title_genres AS G
WHERE T.title_id = G.title_id
AND T.title_type = 'movie'
AND T.start_year <= 2019
GROUP BY T.start_year, G.genre
ORDER BY T.start_year DESC, G.genre ASC;"""

In [147]:
result, column_names = auth_system.execute_query(current_user, Query4)

Executing query: SELECT T.start_year, G.genre, COUNT(DISTINCT T.title_id) AS Number_of_movies
FROM Titles AS T, Title_genres AS G
WHERE T.title_id = G.title_id
AND T.title_type = 'movie'
AND T.start_year <= 2019
GROUP BY T.start_year, G.genre
ORDER BY T.start_year DESC, G.genre ASC;


In [148]:
if result is not None:
    df = pd.DataFrame(result, columns=column_names)
    print(df)

   start_year          genre  Number_of_movies
0        2001  Documentary\r                 1
1        2001    Documentary                 1
2        2001        Short\r                 1
3        2000       Action\r                 2
4        2000        Drama\r                 1
5        1999        Short\r                 1
6        1997      Romance\r                 1
7        1994       Action\r                 1
8        1994       Comedy\r                 1
